### 案例研究

在这个案例研究中，你将使用在上一课中看到的同样的波士顿房屋数据。但是，你现在可以更好地处理此数据集中实际存在的复杂性。

首先，让我们设定库和数据。

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from patsy import dmatrices
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

boston_data = load_boston()
df = pd.DataFrame()
df['MedianHomePrice'] = boston_data.target
df2 = pd.DataFrame(boston_data.data)
df2.columns = boston_data.feature_names
df = df.join(df2)
df.head()

,MedianHomePrice,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,21.6,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,34.7,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,33.4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,36.2,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


你将要用到 notebook 的 **df** 数据帧。你可以在 [这里](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) 找到数据集中关于特征的描述。具体而言，你将尝试使用数据集中的其他变量来构建最佳模型以预测平均房价。

简要摘录：
- CRIM - per capita crime rate by town
- ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS - proportion of non-retail business acres per town.
- CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- NOX - nitric oxides concentration (parts per 10 million)
- RM - average number of rooms per dwelling
- AGE - proportion of owner-occupied units built prior to 1940
- DIS - weighted distances to five Boston employment centres
- RAD - index of accessibility to radial highways
- TAX - full-value property-tax rate per $10,000
- PTRATIO - pupil-teacher ratio by town
- B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT - % lower status of the population
- MEDV - Median value of owner-occupied homes in $1000's

`1.` 使用 [train test_split](http://scikit-learn.org/stable/modules/cross_validation.html) 创建一个 training 数据集与一个 test 数据集，其中20％的数据在 test 数据集中。无序状态设为0。将结果存储在 `X_train, X_test, y_train, y_test` 中。

除非指定，否则只能使用 `X_train` 与 `y_train` 变量来回答以下问题。

`2.` 首先，获取数据集中每个特征的汇总。使用汇总的结果回答下面的第一个与第二个测试题目。同样，使用 [corr](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) 方法对每个变量进行相互比较。

`2.` 接下来，使用 [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) 来缩放数据集中的所有 x 变量。将结果存储在 `X_scaled_train` 中。 创建一个 pandas 数据帧并存储缩放的 x 变量以及 training 响应。把该数据帧命名为 `training_data` 。

`3.` 接下来，记得用 **所有的** 缩放特征来拟合线性模型，以预测此响应（平均房价）。不要忘记添加一个截距。使用你的线性模型的结果来回答下面的第三个测试题目。

`4.` 现在，使用下面的函数来计算数据集中每个 x_variable 的 vif，并使用该函数的结果来回答下面的第四个测试题目。

1. 使用 train test_split 创建一个 training 数据集与一个 test 数据集，其中20％的数据在 test 数据集中。无序状态设为0。将结果存储在 X_train, X_test, y_train, y_test 中

In [2]:
df.head(3)

,MedianHomePrice,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,21.6,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,34.7,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [3]:
df2.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [4]:
X_train, X_test, y_train, y_test = \
train_test_split(df2, df['MedianHomePrice'], test_size = 0.20)

2. 首先，获取数据集中每个特征的汇总。使用汇总的结果回答下面的第一个与第二个测试题目。同样，使用 corr 方法对每个变量进行相互比较。

In [21]:
X_train.shape

(404, 13)

In [19]:
X_train.isnull().count()

CRIM       404
ZN         404
INDUS      404
CHAS       404
NOX        404
RM         404
AGE        404
DIS        404
RAD        404
TAX        404
PTRATIO    404
B          404
LSTAT      404
dtype: int64

In [13]:
X_train.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.609125,11.569307,10.985050,0.071782,0.556484,6.315891,68.556436,3.808195,9.356436,404.032178,18.318317,356.278342,12.457351
std,8.875058,23.152481,6.894618,0.258447,0.117704,0.709452,27.994922,2.131226,8.589721,166.172655,2.228701,91.566533,7.110381
min,0.009060,0.000000,0.740000,0.000000,0.385000,3.863000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.081437,0.000000,5.130000,0.000000,0.452000,5.890500,45.550000,2.087875,4.000000,279.000000,16.800000,375.472500,6.772500
50%,0.261390,0.000000,8.560000,0.000000,0.538000,6.210000,77.700000,3.175750,5.000000,330.000000,18.700000,391.305000,10.925000
75%,3.202962,20.000000,18.100000,0.000000,0.631000,6.636750,93.650000,5.400800,12.000000,666.000000,20.200000,395.755000,16.372500
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [22]:
X_train['CHAS'].mean()

0.07178217821782178

In [6]:
y_train.describe()

count    404.000000
mean      22.796535
std        9.332147
min        5.000000
25%       16.950000
50%       21.600000
75%       26.400000
max       50.000000
Name: MedianHomePrice, dtype: float64

In [23]:
df.corr()

,MedianHomePrice,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
MedianHomePrice,1.000000,-0.388305,0.360445,-0.483725,0.175260,-0.427321,0.695360,-0.376955,0.249929,-0.381626,-0.468536,-0.507787,0.333461,-0.737663
CRIM,-0.388305,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621
ZN,0.360445,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995
INDUS,-0.483725,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800
CHAS,0.175260,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929
NOX,-0.427321,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879
RM,0.695360,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808
AGE,-0.376955,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339
DIS,0.249929,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996
RAD,-0.381626,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676


In [2]:
def vif_calculator(df, response):
    '''
    INPUT:
    df - a dataframe holding the x and y-variables
    response - the column name of the response as a string
    OUTPUT:
    vif - a dataframe of the vifs
    '''
    df2 = df.drop(response, axis = 1, inplace=False)
    features = "+".join(df2.columns)
    y, X = dmatrices(response + ' ~' + features, df, return_type='dataframe')
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["features"] = X.columns
    vif = vif.round(1)
    return vif

`5.` 根据查看 p 值和 VIF 的结果，确定删除 `AGE`、 `NOX` 与 `TAX` ，之后重新开始。用这些已删除的特征（但仍然带有截距）来拟合一个新的线性模型。使用这个线性模型和先期模型的结果来回答下面的第五个测试题目。

`6.` 给出之前的线性回归模型的结果，从模型中删除 `RAD` 变量。然后仔细检查所有的 VIF 是否小于4。所有的变量现在应该显示与响应的具有的线性关系，并且与先前模型相比，Rsquared 值没有发生变化。

`6.` 因为使用线性模型通过 statsmodels 扩展到 test 数据有点乏味，所以我们将使用 [sklearn 这样去做 ](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)。 以下是将完整模型拟合到我们的数据的一个例子。

正如你可以在文档中看到的那样，你需要注意一点，该数据默认使用 sklearn 的线性模型进行缩放。  

我在模型下面的单元格中创建了剩余的 X 矩阵。使用这些来修改并查看哪个模型在 test 集上的效果最好。使用你的结果回答下面的最后一个测试题目。

In [3]:
X_data = df.drop('MedianHomePrice', axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
                 X_data, df['MedianHomePrice'], test_size=0.2, random_state=0)


lm_full = LinearRegression()
lm_full.fit(X_train, y_train)
lm_full.score(X_test, y_test)

0.58920115191864575

In [4]:
X_train_red = X_train.drop(['AGE','NOX','TAX'] , axis=1, inplace=False)
X_test_red = X_test.drop(['AGE','NOX','TAX'] , axis=1, inplace=False)


X_train_red2 = X_train.drop(['AGE','NOX','TAX','RAD'] , axis=1, inplace=False)
X_test_red2 = X_test.drop(['AGE','NOX','TAX','RAD'] , axis=1, inplace=False)


In [5]:
lm_red = LinearRegression()
lm_red.fit(X_train_red, y_train)
print(lm_red.score(X_test_red, y_test))

lm_red2 = LinearRegression()
lm_red2.fit(X_train_red2, y_train)
print(lm_red2.score(X_test_red2, y_test))


0.548663728376
0.545224153736
